In [ ]:
!pip install geocoder
!pip install reverse_geocoder
!pip install shapely

In [110]:
!pip install geocoder
!pip install reverse_geocoder
!pip install shapely
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

import geocoder
import time

import reverse_geocoder as rg

import matplotlib.pyplot as plt


## USING geocoder ( SO SLOW DONT RUN CODE)

In [109]:
#EXEMPLE
g = geocoder.osm([24.147738362622874, -14.87164597884633], method='reverse') 
my_dict = g.json
my_dict

{'accuracy': 0.001,
 'address': "Bir Anzarane ⴱⵉⵔⴰⵏⵣⴰⵔⴰⵏ بئر أنزران, caïdat d'Bir Anzarane, Dakhla-Oued Ed-Dahab ⴷⴰⵅⵍⴰ-ⵡⴰⴷ ⴷⴰⵀⴰⴱ الداخلة-وادي الذهب, Maroc / ⵍⵎⵖⵔⵉⴱ / المغرب",
 'bbox': {'northeast': [24.6832285, -14.0717693],
  'southwest': [23.5545793, -15.4084647]},
 'confidence': 1,
 'country': 'Maroc / ⵍⵎⵖⵔⵉⴱ / المغرب',
 'country_code': 'ma',
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons/poi_boundary_administrative.p.20.png',
 'importance': 0.001,
 'lat': 23.8885639,
 'lng': -14.5313258,
 'municipality': "caïdat d'Bir Anzarane",
 'ok': True,
 'osm_id': 3594878,
 'osm_type': 'relation',
 'place_id': 284133265,
 'place_rank': 16,
 'quality': 'administrative',
 'raw': {'place_id': 284133265,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 3594878,
  'boundingbox': ['23.5545793', '24.6832285', '-15.4084647', '-14.0717693'],
  'lat': '23.8885639',
  'lon': '-14.5313258',
  'display_name': "Bir Anzara

In [2]:
#FUNCTIONS
def reverse_geo(coords):
    city = ' de city '
    region = 'region '
    pays = 'pays '
    for _,(i,j) in enumerate(coords):
        g = geocoder.osm([i,j], method='reverse') 
        my_dict = g.json['raw']['address']
    if 'state_district' in my_dict:
        city = str(my_dict['state_district'])
    if 'region' in my_dict:
        region = str(my_dict['region'])
    if 'country' in my_dict:
        pays = str(my_dict['country'])
    return(city + ',' + region +'@'+ pays)

def get_country(text):
    country = text.split('@')[1]
    country = country.split(' ')[0]
    return country

def get_city(text):
    city = 'city'
    if 'de' in text:
        city = text.split('de ')[1]
    if "d'" in text:
        city = text.split("d'")[1]
    city = city.split(" ")[0]
    return city

def get_region(text):
    region = text.split(",",1)[1]
    region = region.split(' ',1)[0]
    return region

In [ ]:
start = time.time()
data = pd.read_csv('part-00000-tid-8332875100528777397-6a3a4d7b-235c-49d9-b1f1-ce1deeaa1f25-3667-1-c000.csv')
data = data.head(1000)
##GET ALL DATA ABOUT LOCATION
data['raw_info'] = data.apply(lambda x: reverse_geo(x['lat'], x['lng']), axis=1)
#data['city_detect']= data['raw_info'].apply(get_city)

## EXTRACT REGION
data['region']= data['raw_info'].apply(get_region)
end = time.time()
duree = end-start
#data['country']=data['raw_info'].apply(get_country)
#data.drop('raw_info', axis = 1, inplace = True) 

In [12]:
start = time.time()

data = pd.read_csv('part-00000-tid-8332875100528777397-6a3a4d7b-235c-49d9-b1f1-ce1deeaa1f25-3667-1-c000.csv')
data = data.head(1000)
coords = data[['lat', 'lng']].apply(tuple, axis=1).tolist()

results = reverse_geo(coords)
data['raw_info'] = results

end = time.time()
print(end-start)

517.1339647769928


In [16]:
data['region']= data['raw_info'].apply(get_region)

8 mins 36 secs for 1000 data 

3 Hours and half for the whole 24271 dataset

**Conclusion** : High Accuracy, Very Slow ! NOT AN OPTION !

## USING Reverse Geo  Github Repo (NOT ACCURATE)

In [72]:
start = time.time()
data = pd.read_csv('part-00000-tid-8332875100528777397-6a3a4d7b-235c-49d9-b1f1-ce1deeaa1f25-3667-1-c000.csv')
coords = data[['lat', 'lng']].apply(tuple, axis=1).tolist()
results = rg.search(coords, mode=1)
data['new_ville'] = [x['name'] for x in results]
data['region'] = [x['admin1'] for x in results]
end = time.time()

In [73]:
end-start

2.3736424446105957

2.37 seconds for 24271 rows OMG !

In [84]:
count = data.new_ville.value_counts()

count[count<=107].count()

104

In [86]:
data['region'].value_counts()

Grand Casablanca                    5566
Rabat-Sale-Zemmour-Zaer             3426
Marrakech-Tensift-Al Haouz          3303
Tanger-Tetouan                      2171
Fes-Boulemane                       1736
Oriental                            1690
Souss-Massa-Draa                    1409
Meknes-Tafilalet                    1171
Doukkala-Abda                        966
Oued ed Dahab-Lagouira               722
Gharb-Chrarda-Beni Hssen             712
Tadla-Azilal                         475
Chaouia-Ouardigha                    411
Oued Ed-Dahab-Lagouira               254
Taza-Al Hoceima-Taounate             202
                                      43
Guelmim-Es Smara                       6
Laayoune-Boujdour-Sakia El Hamra       3
Ceuta                                  3
Andalusia                              2
Name: region, dtype: int64

**Conclusion** : Good timing, bad accuracy and deprecated region system

## Using Shapefile: (NOT DYNAMIC BUT EFFICIENT)

In [113]:
start = time.time()
# Open the shapefile
regions = gpd.GeoDataFrame.from_file('geojson_MA.json')

data = pd.read_csv('part-00000-tid-8332875100528777397-6a3a4d7b-235c-49d9-b1f1-ce1deeaa1f25-3667-1-c000.csv')
data['coords'] = list(zip(data['lng'],data['lat']))
data['coords'] = data['coords'].apply(Point)
datageo = gpd.GeoDataFrame(data, geometry='coords', crs=regions.crs)

# Perform spatial join to match points and polygons
datageo = gpd.tools.sjoin(datageo, regions, predicate="within", how='left')
datageo.drop(columns={'coords','index_right','cartodb_id'},inplace=True)
end = time.time()

In [114]:
end-start

2.4595701694488525

In [115]:
datageo

,categoryName,title,lat,lng,city,reviewsCount,source,category,region
0,Magasin de cosmétiques,Medipromo,35.752776,-5.831695,Tanger,16.0,Google_places,BoutiqueOrLocalShop,Tanger-Tétouan-Al Hoceïma
1,Pharmacie,Pharmacie de la Santé,35.758273,-5.801484,Tanger,5.0,Google_places,Pharmacy,Tanger-Tétouan-Al Hoceïma
2,Mosque,MOSQUEE AL FAJR,34.661905,-1.888478,Oujda,27.0,Google_places,Mosque,L'Oriental
3,Pharmacie,Pharmacie de la croix d'or,33.585810,-7.638556,Casablanca,10.0,Google_places,Pharmacy,Casablanca-Settat
4,Restaurant,Restaurant Les Blancs,30.423927,-9.615397,Agadir,666.0,Google_places,Restaurant,Souss-Massa
...,...,...,...,...,...,...,...,...,...
24266,Café et restaurant de grillades,Cafe Snack Rahba Kedima,31.628858,-7.986880,Marrakech,95.0,Google_places,CafeOrTeahouse,Marrakech-Safi
24267,Restaurant,Cabaret Cavalli Oriental marrakech sky armani,31.626928,-8.017685,Marrakech,5.0,Google_places,Restaurant,Marrakech-Safi
24268,Restaurant marocain,Abdul Rahman,34.064519,-4.978933,Fès,37.0,Google_places,Restaurant,Fès-Meknès
24269,Restaurant,Soussi Restaurant,35.786443,-5.812002,Tanger,29.0,Google_places,Restaurant,Tanger-Tétouan-Al Hoceïma


2.16 Seconds for 24271 row ! LET'S GO !

**conclusion** : Not Dynamic, but Efficient ! 